# Few-shot prompting with PDFs and Gemini

This notebook shows how to use few-shot prompting with multimodal inputs, specifically PDFs, with the Gemini API. Few-shot prompting is a powerful technique where you provide examples of the task you want the model to perform within the prompt. By showing the model a few examples of inputs and desired outputs, you can guide it to generate more accurate and contextually appropriate responses.

This is achieved by constructing a series of `Content` objects with alternating `"user"` and `"model"` roles.

Let's get started!

In [ ]:
from google import genai
import httpx
import io
from google.genai.types import Content, FileData, GenerateContentConfig, Part

client = genai.Client()


def upload_pdf(file_url: str) -> Part:
    doc_io = io.BytesIO(httpx.get(file_url).content)
    file = client.files.upload(
        file=doc_io,
        config=dict(mime_type='application/pdf')
    )
    return Part(file_data=FileData(mime_type=file.mime_type, file_uri=file.uri))


example_files = [
    "https://arxiv.org/pdf/2503.10622",
    "https://arxiv.org/pdf/2503.10613",
]

example_outputs = [
    "This paper is about Transformers without normalization.",
    "This paper is about improving text-to-image models for multi-turn image editing.",
]

text_part = Part(text="Provide a one-sentence summary of this paper")

messages = []

# add few-shot examples
for file_url, output in zip(example_files, example_outputs):
    pdf_part = upload_pdf(file_url)
    messages.extend(
        [
            Content(role="user", parts=[pdf_part, text_part]),
            Content(role="model", parts=[Part(text=output)]),
        ]
    )

# add new query
pdf_url = "https://arxiv.org/pdf/2503.10633"
pdf_part = upload_pdf(pdf_url)
messages.append(
    Content(role="user", parts=[pdf_part, text_part])
)

system_prompt = "You are a helpful assistant that can generate one-line summaries of papers."
response = client.models.generate_content(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(system_instruction=system_prompt),
    contents=messages,
)

print(response.text)